In [1]:
import pandas as pd

# 파일 경로 설정
file_21 = '/Users/hayden/Desktop/2021년전체측정소자료.csv'
file_22 = '/Users/hayden/Desktop/2022년전체측정소자료.csv'
file_23 = '/Users/hayden/Desktop/2023년전체측정소자료.csv'

# 각각의 CSV 파일을 데이터프레임으로 읽기
df_21 = pd.read_csv(file_21)
df_22 = pd.read_csv(file_22)
df_23 = pd.read_csv(file_23)

# 데이터프레임을 리스트에 담기
dfs = [df_21, df_22, df_23]

# concat 함수를 사용하여 합치기
df = pd.concat(dfs, ignore_index=True)

# 결과 확인
print(df.head())  # 합쳐진 데이터프레임의 첫 부분 출력


       MSR_DT STTN_NM  PM25  PM10     O3   CO    SO2    NO2     NO    NOX
0  2021010101     명서동  11.0  22.0  0.022  0.5  0.003  0.023  0.009  0.032
1  2021010102     명서동  12.0  16.0  0.028  0.4  0.003  0.015  0.007  0.022
2  2021010103     명서동  13.0  21.0  0.031  0.4  0.003  0.013  0.006  0.019
3  2021010104     명서동  15.0  19.0  0.028  0.4  0.003  0.013  0.006  0.019
4  2021010105     명서동  15.0  22.0  0.024  0.4  0.003  0.015  0.006  0.021


In [2]:
df = df[~df['STTN_NM'].isin(['이동차'])]
df.drop(columns=['NO', 'NOX'], inplace=True)
print(df)

# 110만개 

             MSR_DT STTN_NM  PM25  PM10     O3   CO    SO2    NO2
0        2021010101     명서동  11.0  22.0  0.022  0.5  0.003  0.023
1        2021010102     명서동  12.0  16.0  0.028  0.4  0.003  0.015
2        2021010103     명서동  13.0  21.0  0.031  0.4  0.003  0.013
3        2021010104     명서동  15.0  19.0  0.028  0.4  0.003  0.013
4        2021010105     명서동  15.0  22.0  0.024  0.4  0.003  0.015
...             ...     ...   ...   ...    ...  ...    ...    ...
1122339  2023123120     부산항   0.0   0.0  0.038  0.4  0.002  0.011
1122340  2023123121     부산항   0.0   0.0  0.037  0.4  0.001  0.010
1122341  2023123122     부산항   0.0   0.0  0.035  0.4  0.001  0.012
1122342  2023123123     부산항   0.0   0.0  0.035  0.4  0.001  0.010
1122343  2023123124     부산항   0.0   0.0  0.030  0.4  0.001  0.012

[1104154 rows x 8 columns]


In [3]:
import pandas as pd
import numpy as np

# 결측치가 있는 열을 선형 보간으로 채우기
for col in df.columns:
    # if df[col].dtype != 'datetime64[ns]':
    df[col] = df[col].interpolate(method='linear', limit_direction='both')

# DataFrame 복사본 생성 및 NULL 값 개수 확인
df_copy = df.copy()

# 각 컬럼의 NULL 값 개수 확인
null_counts = df_copy.isnull().sum()

# 특정 컬럼의 NULL 값 개수 확인
PM25_null_count = df_copy['PM25'].isnull().sum()
PM10_null_count = df_copy['PM10'].isnull().sum()
O3_null_count = df_copy['O3'].isnull().sum()
CO_null_count = df_copy['CO'].isnull().sum()
SO2_null_count = df_copy['SO2'].isnull().sum()
NO2_null_count = df_copy['NO2'].isnull().sum()

print("PM25 컬럼의 NULL 값 개수:", PM25_null_count)
print("PM10 컬럼의 NULL 값 개수:", PM10_null_count)
print("O3 컬럼의 NULL 값 개수:", O3_null_count)
print("CO 컬럼의 NULL 값 개수:", CO_null_count)
print("SO2 컬럼의 NULL 값 개수:", SO2_null_count)
print("NO2 컬럼의 NULL 값 개수:", NO2_null_count)

# 전체 데이터프레임의 NULL 값 개수 확인
total_null_count = df_copy.isnull().sum().sum()
print("전체 데이터프레임의 NULL 값 개수:", total_null_count)

/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_74943/140698646.py:7: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df[col] = df[col].interpolate(method='linear', limit_direction='both')


PM25 컬럼의 NULL 값 개수: 0
PM10 컬럼의 NULL 값 개수: 0
O3 컬럼의 NULL 값 개수: 0
CO 컬럼의 NULL 값 개수: 0
SO2 컬럼의 NULL 값 개수: 0
NO2 컬럼의 NULL 값 개수: 0
전체 데이터프레임의 NULL 값 개수: 0


In [4]:
import pandas as pd
import numpy as np
import altair as alt
import base64
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 데이터 로드
# df_sambang = pd.read_csv('your_data.csv')

# 비수치 데이터 제거
df_numeric = df.drop(columns=['STTN_NM'])

# 데이터 전처리
df_numeric['MSR_DT'] = pd.to_datetime(df_numeric['MSR_DT'], format='%Y%m%d%H', errors='coerce')
df_numeric['Year'] = df_numeric['MSR_DT'].dt.year
df_numeric['Month'] = df_numeric['MSR_DT'].dt.month
df_numeric['Hour'] = df_numeric['MSR_DT'].dt.hour

# MinMaxScaler를 사용하여 정규화
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df_numeric[numeric_columns] = scaler.fit_transform(df_numeric[numeric_columns])

# StandardScaler를 사용하여 표준화
standard_scaler = StandardScaler()
df_numeric[numeric_columns] = standard_scaler.fit_transform(df_numeric[numeric_columns])

# 월별 평균 계산
monthly_avg = df_numeric.groupby('Month')[numeric_columns].mean().reset_index()
monthly_avg_melted = monthly_avg.melt(id_vars=['Month'], value_vars=numeric_columns, var_name='category', value_name='value')

# 연도별 평균 계산
yearly_avg = df_numeric.groupby('Year')[numeric_columns].mean().reset_index()
yearly_avg_melted = yearly_avg.melt(id_vars=['Year'], value_vars=numeric_columns, var_name='category', value_name='value')

# 시간별 평균 계산
hourly_avg = df_numeric.groupby('Hour')[numeric_columns].mean().reset_index()
hourly_avg_melted = hourly_avg.melt(id_vars=['Hour'], value_vars=numeric_columns, var_name='category', value_name='value')

# 상관 관계 계산 (상관 분석)
df_sambang_corr = df_numeric.drop(columns=['MSR_DT', 'Year', 'Month', 'Hour']).corr().reset_index().melt(id_vars='index')

# Altair의 행 수 제한을 늘림
alt.data_transformers.disable_max_rows()

# Altair 차트 생성
chart_width = 300
chart_height = 200

# 1. 시간에 따른 표준화된 대기질 측정값
time_chart = alt.Chart(df_sambang_numeric).transform_fold(
    numeric_columns,
    as_=['category', 'value']
).mark_line().encode(
    x='MSR_DT:T',
    y='value:Q',
    color='category:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='Standardized Air Quality Measurements Over Time'
)

# 2. 월별 대기질 지표 평균
monthly_chart = alt.Chart(monthly_avg_melted).mark_bar().encode(
    x='Month:O',
    y='value:Q',
    color='category:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='Monthly Average of Air Quality Indicators'
)

# 3. 연도별 대기질 지표 평균
yearly_chart = alt.Chart(yearly_avg_melted).mark_bar().encode(
    x='Year:O',
    y='value:Q',
    color='category:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='Yearly Average of Air Quality Indicators'
)

# 4. 시간별 대기질 지표 평균
hourly_chart = alt.Chart(hourly_avg_melted).mark_line(point=True).encode(
    x='Hour:O',
    y='value:Q',
    color='category:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='Hourly Average of Air Quality Indicators'
)

# 5. 상관 행렬 히트맵
corr_chart = alt.Chart(df_sambang_corr).mark_rect().encode(
    x='index:O',
    y='variable:O',
    color='value:Q'
).properties(
    width=chart_width,
    height=chart_height,
    title='Correlation Matrix'
)

# 로컬 이미지 파일을 base64로 인코딩
image_path = '/Users/hayden/Desktop/Screenshot 2024-07-09 at 3.33.03 PM.png'  # 로컬 이미지 경로
with open(image_path, 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

image_url = f'data:image/png;base64,{image_base64}'

# Altair 이미지 차트 생성
image_chart = alt.Chart(pd.DataFrame({'url': [image_url]})).mark_image(
    width=chart_width,
    height=chart_height,
).encode(
    url='url:N'
).properties(
    title='김해시 삼방동 측정소 : 김해시 활천로 303(신어초등학교상)'
)

# 모든 차트를 함께 표시
final_chart = alt.vconcat(
    image_chart,
    alt.hconcat(time_chart, monthly_chart, yearly_chart),
    alt.hconcat(hourly_chart, corr_chart)
).resolve_legend(
    color="independent",
    size="independent"
).resolve_scale(
    x="independent",
    y="independent"
).resolve_axis(
    x="independent",
    y="independent"
)

final_chart.display()


NameError: name 'df_sambang_numeric' is not defined

In [ ]:
# MinMaxScaler를 사용하여 정규화
scaler = MinMaxScaler()
df_sambang[numeric_columns] = scaler.fit_transform(df_sambang[numeric_columns])

# StandardScaler를 사용하여 표준화
standard_scaler = StandardScaler()
df_sambang_numeric[numeric_columns] = standard_scaler.fit_transform(df_sambang_numeric[numeric_columns])

/var/folders/6k/r_smdd550yddqpmrhzs0gqyh0000gn/T/ipykernel_57524/370144706.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sambang[numeric_columns] = scaler.fit_transform(df_sambang[numeric_columns])


In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import base64
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 데이터 전처리
df_sambang_numeric = df_sambang.drop(columns=['STTN_NM'])
df_sambang_numeric['MSR_DT'] = pd.to_datetime(df_sambang_numeric['MSR_DT'], format='%Y%m%d%H', errors='coerce')
df_sambang_numeric['Year'] = df_sambang_numeric['MSR_DT'].dt.year
df_sambang_numeric['Month'] = df_sambang_numeric['MSR_DT'].dt.month
df_sambang_numeric['Hour'] = df_sambang_numeric['MSR_DT'].dt.hour

# MinMaxScaler를 사용하여 정규화
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df_sambang_numeric[numeric_columns] = scaler.fit_transform(df_sambang_numeric[numeric_columns])

# StandardScaler를 사용하여 표준화
standard_scaler = StandardScaler()
df_sambang_numeric[numeric_columns] = standard_scaler.fit_transform(df_sambang_numeric[numeric_columns])

# 월별 평균 계산
monthly_avg = df_sambang.groupby('Month')[['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']].mean().reset_index()
source = monthly_avg.melt(id_vars=['Month'], 
                          value_vars=['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2'],
                          var_name='category', 
                          value_name='value')
# 연도별 평균 계산
yearly_avg = df_sambang.groupby('Year')[numeric_columns].mean().reset_index()
source2 = yearly_avg.melt(id_vars=['Year'], 
                          value_vars=['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2'],
                          var_name='category', 
                          value_name='value')
# 시간별 평균 계산
hourly_avg = df_sambang.groupby('Hour')[numeric_columns].mean().reset_index()
hourly_avg_melted = hourly_avg.melt(id_vars=['Hour'], value_vars=numeric_columns, var_name='category', value_name='value')

# 상관 관계 계산 (상관 분석)
df_sambang_corr = df_sambang.drop(columns=['MSR_DT', 'Year', 'Month', 'Hour'])
corr_matrix = df_sambang_corr.corr(numeric_only=True)
# 상관 행렬을 DataFrame으로 변환
corr_matrix = corr_matrix.reset_index().melt(id_vars='index')

# Altair의 행 수 제한을 늘림
alt.data_transformers.disable_max_rows()

# Altair 차트 생성
chart_width = 300
chart_height = 300

# 1. 시간에 따른 표준화된 대기질 측정값
time_chart = alt.Chart(df_sambang_numeric).transform_fold(
    numeric_columns,
    as_=['category', 'value']
).mark_line().encode(
    x='MSR_DT:T',
    y='value:Q',
    color='category:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='Standardized Air Quality Measurements Over Time'
)

# 2. 월별 대기질 지표 평균
monthly_chart = alt.Chart(source).mark_bar().encode(
    alt.X('Month:O', title='Month'),
    alt.Y('value:Q', title='Average Value'),
    alt.Color('category:N', title='Category')
).properties(
    width=chart_width,
    height=chart_height,
    title='Monthly Average of Air Quality Indicators'
)


# 3. 연도별 대기질 지표 평균
yearly_chart = alt.Chart(source2).mark_bar().encode(
    x='Year:O',
    y='value:Q',
    color='category:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='Yearly Average of Air Quality Indicators'
)

# 4. 시간별 대기질 지표 평균
hourly_chart = alt.Chart(hourly_avg_melted).mark_line(point=True).encode(
    x='Hour:O',
    y='value:Q',
    color=alt.Color('category:N', legend=None)  # 범례 제거
).properties(
    width=chart_width,
    height=chart_height,
    title='Hourly Average of Air Quality Indicators'
)

# 5. 상관 행렬 히트맵

# 상관 행렬 차트 생성
base = alt.Chart(corr_matrix).encode(
    x='index:O',
    y='variable:O'
)

# 상관 행렬 히트맵
heatmap = base.mark_rect().encode(
    color='value:Q'
)

# 상관 행렬 값 텍스트
text = base.mark_text(baseline='middle').encode(
    text=alt.Text('value:Q', format='.2f'),
    color=alt.condition(
        alt.datum.value > 0.5,  # 조건에 따라 텍스트 색상 변경
        alt.value('black'),
        alt.value('white')
    )
)
# 차트 결합
corr_chart = (heatmap + text).properties(
    width=chart_width,
    height=chart_height,
    title='Correlation Matrix'
)

# 로컬 이미지 파일을 base64로 인코딩
image_path = '/Users/hayden/Desktop/Screenshot 2024-07-09 at 3.33.03 PM.png'  # 로컬 이미지 경로
with open(image_path, 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

image_url = f'data:image/png;base64,{image_base64}'

# Altair 이미지 차트 생성
image_chart = alt.Chart(pd.DataFrame({'url': [image_url]})).mark_image(
    width=chart_width,
    height=chart_height,
).encode(
    url='url:N'
).properties(
    title='김해시 삼방동 측정소 : 김해시 활천로 303(신어초등학교상)'
)

# 모든 차트를 함께 표시
spacer = alt.Chart().mark_rect(color='white').encode(
    height=10  # 공백 차트의 높이 설정
)

final_chart = alt.vconcat(
    alt.hconcat(image_chart, time_chart),
    alt.hconcat(monthly_chart, yearly_chart),
    alt.hconcat(hourly_chart, corr_chart)
).configure_concat(
    spacing=20,  # 차트 간의 간격을 설정
    # titlePadding=10,  # 제목과 차트 간의 간격 설정
    # titleAlign="left"  # 제목 정렬 설정

).resolve_legend(
    color="independent",
    size="independent"
).resolve_scale(
    x="independent",
    y="independent"
).resolve_axis(
    x="independent",
    y="independent"
)

final_chart.display()



KeyError: 'Month'

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import base64
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 데이터 전처리
df_sambang_numeric = df_sambang.drop(columns=['STTN_NM'])
df_sambang_numeric['MSR_DT'] = pd.to_datetime(df_sambang_numeric['MSR_DT'], format='%Y%m%d%H', errors='coerce')
df_sambang_numeric['Year'] = df_sambang_numeric['MSR_DT'].dt.year
df_sambang_numeric['Month'] = df_sambang_numeric['MSR_DT'].dt.month
df_sambang_numeric['Hour'] = df_sambang_numeric['MSR_DT'].dt.hour

# MinMaxScaler를 사용하여 정규화
scaler = MinMaxScaler()
numeric_columns = ['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']
df_sambang_numeric[numeric_columns] = scaler.fit_transform(df_sambang_numeric[numeric_columns])

# StandardScaler를 사용하여 표준화
standard_scaler = StandardScaler()
df_sambang_numeric[numeric_columns] = standard_scaler.fit_transform(df_sambang_numeric[numeric_columns])

# 월별 평균 계산
monthly_avg = df_sambang.groupby('Month')[['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2']].mean().reset_index()
source = monthly_avg.melt(id_vars=['Month'], 
                          value_vars=['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2'],
                          var_name='category', 
                          value_name='value')
# 연도별 평균 계산
yearly_avg = df_sambang.groupby('Year')[numeric_columns].mean().reset_index()
source2 = yearly_avg.melt(id_vars=['Year'], 
                          value_vars=['PM25', 'PM10', 'O3', 'CO', 'SO2', 'NO2'],
                          var_name='category', 
                          value_name='value')
# 시간별 평균 계산
hourly_avg = df_sambang.groupby('Hour')[numeric_columns].mean().reset_index()
hourly_avg_melted = hourly_avg.melt(id_vars=['Hour'], value_vars=numeric_columns, var_name='category', value_name='value')

# 상관 관계 계산 (상관 분석)
df_sambang_corr = df_sambang.drop(columns=['MSR_DT', 'Year', 'Month', 'Hour'])
corr_matrix = df_sambang_corr.corr(numeric_only=True)
# 상관 행렬을 DataFrame으로 변환
corr_matrix = corr_matrix.reset_index().melt(id_vars='index')

# Altair의 행 수 제한을 늘림
alt.data_transformers.disable_max_rows()

# Altair 차트 생성
chart_width = 300
chart_height = 300

# 1. 시간에 따른 표준화된 대기질 측정값
time_chart = alt.Chart(df_sambang_numeric).transform_fold(
    numeric_columns,
    as_=['category', 'value']
).mark_line().encode(
    x='MSR_DT:T',
    y='value:Q',
    color='category:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='Standardized Air Quality Measurements Over Time'
)

# 2. 월별 대기질 지표 평균
monthly_chart = alt.Chart(source).mark_bar().encode(
    alt.X('Month:O', title='Month'),
    alt.Y('value:Q', title='Average Value'),
    alt.Color('category:N', title='Category')
).properties(
    width=chart_width,
    height=chart_height,
    title='Monthly Average of Air Quality Indicators'
)

# 3. 연도별 대기질 지표 평균
yearly_chart = alt.Chart(source2).mark_bar().encode(
    x='Year:O',
    y='value:Q',
    color='category:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='Yearly Average of Air Quality Indicators'
)

# 4. 시간별 대기질 지표 평균
hourly_chart = alt.Chart(hourly_avg_melted).mark_line(point=True).encode(
    x='Hour:O',
    y='value:Q',
    color=alt.Color('category:N', legend=None)  # 범례 제거
).properties(
    width=chart_width,
    height=chart_height,
    title='Hourly Average of Air Quality Indicators'
)

# 5. 상관 행렬 히트맵
base = alt.Chart(corr_matrix).encode(
    x='index:O',
    y='variable:O'
)

heatmap = base.mark_rect().encode(
    color='value:Q'
)

text = base.mark_text(baseline='middle').encode(
    text=alt.Text('value:Q', format='.2f'),
    color=alt.condition(
        alt.datum.value > 0.5,
        alt.value('black'),
        alt.value('white')
    )
)

corr_chart = (heatmap + text).properties(
    width=chart_width,
    height=chart_height,
    title='Correlation Matrix'
)

# 로컬 이미지 파일을 base64로 인코딩
image_path = '/Users/hayden/Desktop/Screenshot 2024-07-09 at 3.33.03 PM.png'
with open(image_path, 'rb') as f:
    image_base64 = base64.b64encode(f.read()).decode('utf-8')

image_url = f'data:image/png;base64,{image_base64}'

# Altair 이미지 차트 생성
image_chart = alt.Chart(pd.DataFrame({'url': [image_url]})).mark_image(
    width=chart_width,
    height=chart_height,
).encode(
    url='url:N'
).properties(
    width=chart_width,
    height=chart_height,
    title='김해시 삼방동 측정소 : 김해시 활천로 303(신어초등학교상)'
)

# 첫 줄의 두 차트를 나란히 배치
first_row = alt.hconcat(image_chart, time_chart)

# 모든 차트를 함께 표시
final_chart = alt.vconcat(
    first_row,
    alt.hconcat(monthly_chart, yearly_chart),
    alt.hconcat(hourly_chart, corr_chart)
).configure_concat(
    spacing=20
).resolve_legend(
    color="independent",
    size="independent"
).resolve_scale(
    x="independent",
    y="independent"
).resolve_axis(
    x="independent",
    y="independent"
)

final_chart.display()

# by park

KeyError: 'Month'